In [1]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

# Prompt+LLM
基本构成： 
PromptTemplate / ChatPromptTemplate -> LLM / ChatModel -> OutputParser

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("给我讲一个关于{foo}的笑话")
model = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)
chain = prompt | model

In [3]:
chain.invoke({"foo": "狗熊"})

AIMessage(content='好的，这是一个关于狗熊的笑话：\n\n有一天，一只狗熊走进了一家酒吧。他走到吧台前，对酒保说：“请给我一杯啤酒。”\n\n酒保看着狗熊，惊讶地问道：“你会喝酒吗？”\n\n狗熊回答道：“当然会！我是狗熊啊！”\n\n酒保犹豫了一下，然后说：“好吧，那你给我证明一下。”\n\n狗熊点了点头，然后开始喝啤酒。他一口气喝下了整整一杯，然后把杯子放在吧台上。\n\n酒保惊讶地说：“哇，你真的会喝酒！”\n\n狗熊笑了笑，说：“当然了，我是狗熊啊！”\n\n酒保好奇地问：“那你能不能再喝一杯？”\n\n狗熊点了点头，然后又一口气喝下了第二杯啤酒。\n\n酒保惊讶地说：“太厉害了！你真的是个会喝酒的狗熊！”\n\n狗熊笑了笑，说：“当然了，我是狗熊啊！”\n\n酒保继续问：“那你能不能再喝一杯？”\n\n狗熊点了点头，然后又一口气喝下了第三杯啤酒。\n\n酒保惊讶地说：“你真的是个了不起的狗熊！”\n\n狗熊笑了笑，说：“当然了，我是狗熊啊！”\n\n酒保好奇地问：“那你能不能再喝一杯？”\n\n狗熊摇了摇头，说：“不行了，我得回家了。”\n\n酒保奇怪地问：“为什么？你不是说你是个会喝酒的狗熊吗？”\n\n狗熊笑了笑，说：“是啊，我是个会喝酒的狗熊，但我妈妈说，喝酒后不准开车。”')

自定义停止输出符

In [4]:
chain = prompt | model.bind(stop=["\n"])

In [5]:
chain.invoke({"foo": "狗熊"})

AIMessage(content='好的，这是一个关于狗熊的笑话：')

兼容openai函数调用的方式

In [6]:
functions = [
    {
        "name": "joke",
        "description": "讲笑话",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "笑话的开头"},
                "punchline": {
                    "type": "string",
                    "description": "爆梗的结尾",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)

In [7]:
chain.invoke({"foo": "男人"}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "setup": "为什么男人总是喜欢开玩笑？",\n  "punchline": "因为他们觉得自己是个笑话！"\n}', 'name': 'joke'}})

输出解析器

In [8]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()

In [9]:
chain.invoke({"foo": "女人"})

'好的，这是一个关于女人的笑话：\n\n有一天，一个女人走进一家宠物店，她对店主说：“我想买一只特别聪明的宠物。”店主笑着回答：“我有一只非常聪明的鹦鹉，它会说七种语言。”女人很兴奋地问：“真的吗？那你能让它给我说说吗？”店主点了点头，然后对鹦鹉说：“鹦鹉，给这位女士说说你会的语言。”鹦鹉立刻回答：“当然！你好，你好，你好，你好，你好，你好，你好！”女人有些失望地说：“这只鹦鹉只会说一句话。”店主笑着说：“是的，但是它会用七种不同的语言说。”'

与函数调用混合使用

In [10]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)

In [11]:
chain.invoke({"foo": "女人"})

{'setup': '为什么女人喜欢逛街？', 'punchline': '因为逛街是唯一能让她们花钱时不觉得心疼的事情。'}

In [17]:
#只输出setup
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

chain = (
    {"foo": RunnablePassthrough()} #使用RunnablePassthrough()跳过prompt
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="punchline") # 定义输出的key
)

In [18]:
#chain.invoke({"foo": "女人"})
#使用RunnablePassthrough()跳过prompt
chain.invoke("女人")

'因为打折时可以一举两得，既能减肥又能增加衣柜！'